In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return (f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
#train_text = train_text[ : 128]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [5]:
bigramDict = dict()
totChar = set(text)
bigramList = list()
count = 0;
for i in totChar:
    for j in totChar:
        bigramDict[i+j] = count
        bigramList.append(i+j)
        count += 1
        pass
    pass
print(count)
print(bigramDict['ax'])

729
52


In [6]:
vocabulary_size = len(bigramDict)

def Bigram2id(bg):
  return bigramDict[bg]
  
def id2Bigram(id):
  return bigramList[id]

print(Bigram2id('ax'), Bigram2id('zs'), Bigram2id('aa'), Bigram2id('as'))
print(id2Bigram(1), id2Bigram(26), id2Bigram(0))

52 720 28 45
 a  z   


In [7]:
import collections
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)

  for _ in range(span):
    bg = text[data_index];
    data_index = (data_index+1)%len(text);
    bg = bg+text[data_index];
    #data_index = (data_index+1)%len(text);
    buffer.append(bigramDict[bg])
    
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    bg = text[data_index];
    data_index = (data_index+1)%len(text);
    bg = bg+text[data_index];
    #data_index = (data_index+1)%len(text);
    buffer.append(bigramDict[bg])
  return batch, labels


print(text[0:10]);
for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [bigramList[bi] for bi in batch])
    print('    labels:', [bigramList[li] for li in labels.reshape(8)])

 anarchism

with num_skips = 2 and skip_window = 1:
    batch: ['an', 'an', 'na', 'na', 'ar', 'ar', 'rc', 'rc']
    labels: ['na', ' a', 'ar', 'an', 'rc', 'na', 'ch', 'ar']

with num_skips = 4 and skip_window = 2:
    batch: ['na', 'na', 'na', 'na', 'ar', 'ar', 'ar', 'ar']
    labels: [' a', 'rc', 'ar', 'an', 'na', 'rc', 'an', 'ch']


In [8]:
import math

embd_batch_size = 126
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 2 # How many words to consider left and right.
num_skips = 3 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():
  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[embd_batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[embd_batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
  # Variables.
  embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0,1.0))
  softmax_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
    stddev=1.0 / math.sqrt(embedding_size)))
  softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset)
  # Compute the softmax loss, using a sample of the negative labels each time.
  loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases, embed,train_labels, num_sampled, vocabulary_size))

  # Optimizer.
  optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
  # Compute the similarity between minibatch examples and all embeddings.
  # We use the cosine distance:
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
  similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [ ]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  average_loss = 0
  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(
      embd_batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if False:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = bigramList[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = 'Nearest to %s:' % valid_word
        for k in xrange(top_k):
          close_word = bigramList[nearest[k]]
          log = '%s %s,' % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()

## final_embeddings is the embeddings of the bigrams 

In [ ]:
batch_size=64
num_unrollings=10

print(final_embeddings.shape)

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, embedding_size), dtype=np.float32)
    for b in range(self._batch_size):
      bg= self._text[self._cursor[b]];
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
      bg += self._text[self._cursor[b]]
      #self._cursor[b] = (self._cursor[b] + 1) % self._text_size
      batch[b] = final_embeddings[Bigram2id(bg)]
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  #print(probabilities.shape)
  #tmp = np.matmul(probabilities,np.transpose(final_embeddings));
  
  tmpvalue = np.dot(probabilities,np.transpose(final_embeddings))/ np.linalg.norm(probabilities)

  return [id2Bigram(c) for c in np.argmax(tmpvalue, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, [x[0] for x in characters(b)])]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

#print((train_batches.next()))
print(batches2string(train_batches.next()))
#print(batches2string(valid_batches.next()))
#print(batches2string(valid_batches.next()))

In [ ]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  #return np.argmax(distribution)
  #distribution = np.array([i for i in distribution])
  #print(distribution)
  #distribution = (distribution/np.sum(distribution, 0)[:,None])
  #print(distribution.shape)
  
  norm_dist = (distribution- np.min(distribution))/(np.max(distribution)-np.min(distribution));
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(norm_dist)):
    s += norm_dist[i]
    if s >= r:
      return i
  return len(norm_dist) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, embedding_size], dtype=np.float)

  tmpvalue = np.dot(prediction,np.transpose(final_embeddings))/ np.linalg.norm(prediction)

  q = sample_distribution(tmpvalue[0])
  p[0] = final_embeddings[q]
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, embedding_size])
  print(b.shape)
  return (b/np.sum(b, 1)[:,None])



In [ ]:
num_nodes = 64
totalGate = 4
forgetGateIndex = 0
inputGateIndex = 1
updateGateIndex = 2
outputGateIndex = 3

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, embedding_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([embedding_size]))
    
  gateWeightInput = tf.Variable(tf.truncated_normal([totalGate, embedding_size , num_nodes] , -0.1 , 0.1));
  gateWeightOutput= tf.Variable(tf.truncated_normal([totalGate, num_nodes , num_nodes] , -0.1 , 0.1));
  gateBias = tf.Variable(tf.zeros([totalGate , 1, num_nodes]));
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    
    stackedInput = tf.pack([i for _ in range(totalGate)]);
    stackedOutput = tf.pack([o for _ in range(totalGate)])
    
    stackedAll = tf.batch_matmul(stackedInput,gateWeightInput)+tf.batch_matmul(stackedOutput,gateWeightOutput)+gateBias
    
    listOfGates = tf.unpack(stackedAll)
    
    forgetGate = tf.sigmoid(listOfGates[forgetGateIndex])
    inputGate = tf.sigmoid(listOfGates[inputGateIndex])
    updateGate = tf.tanh(listOfGates[updateGateIndex])
    outputGate = tf.sigmoid(listOfGates[outputGateIndex])
    
    state = forgetGate * state + inputGate * updateGate
    
    return outputGate*tf.tanh(state),state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,embedding_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))
    #loss  = tf.reduce_mean(tf.nn.l2_loss(logits-tf.concat(0,train_labels)))
    alpha = 0.0005
    #loss += alpha * (tf.nn.l2_loss(gateWeightInput) + tf.nn.l2_loss(gateWeightOutput))
    loss += alpha * tf.nn.l2_loss(w)

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(2.0, global_step, 5000, 0.95, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, embedding_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [ ]:
import time
cur_time = time.time()

num_steps = 10001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      #print(labels.shape)
      #print(predictions.shape)
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency*10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          #print(feed)
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            #print(prediction)
            feed = sample(prediction)
            sentence += characters(feed)[0][1]
            #print(sentence)
            #print(feed)
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))
    
cur_time = time.time()-cur_time
print(cur_time)